# ControlVideo

### Installation

Only for colab

In [ ]:
!git clone --recurse-submodules https://github.com/rossiyareich/marching-waifu-x.git
%cd marching-waifu

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Only for local install (run `setup.sh` or `setup.bat` and `huggingface-cli login` beforehand!)

In [ ]:
%cd ../

Project setup

In [ ]:
!nvidia-smi

# Install requirements
!python -m pip install -r requirements.txt
!python -m pip install -r ext/Real-ESRGAN/requirements.txt

# Create directories
!mkdir data/dataset/nerf/original/
!mkdir data/dataset/nerf/train/

# Install local packages
%cd ext/Real-ESRGAN/
!python setup.py develop
%cd ../../

In [ ]:
%cd scripts

In [ ]:
import sys
sys.path.append("..")

### Configuration

In [ ]:
#@title Inference configuration
class config(object):
    pass

#@markdown ###**ControlVideo paths**
controlvideo = config()
controlvideo.out_path = "../data/dataset/nerf/original/"  #@param {type:"string"}
controlvideo.sd_repo = "rossiyareich/Nabylon-v1.0-fp16"  #@param {type:"string"}
controlvideo.vae_repo = "stabilityai/sd-vae-ft-mse"  #@param {type:"string"}
controlvideo.controlnet_repo = "lllyasviel/control_v11p_sd15_openpose"  #@param {type:"string"}
controlvideo.ifnet_path = "../data/checkpoints/flownet.pkl"  #@param {type:"string"}
controlvideo.cache_dir = "../data/checkpoints/"  #@param {type:"string"}
#@markdown ###**ControlVideo**
controlvideo.prompt = "(masterpiece, best quality)+, 1girl, white hoodie, earmuffs, leggings, white scarf, black gloves, white socks, short blue hair, blue eyes, bangs"  #@param {type:"string"}
controlvideo.negative_prompt = "EasyNegative, (worst quality, low quality, logo, text, watermark, username, nsfw), inaccurate hands and fingers"  #@param {type:"string"}
controlvideo.textual_inversion_path = "../data/embeddings/"  #@param {type:"string"}
controlvideo.controlnet_conditioning_path = "../data/dataset/conditioning/"  #@param {type:"string"}
controlvideo.video_length = 15  #@param {type:"slider", min:1, max:250, step:1}
controlvideo.num_inference_steps = 50  #@param {type:"slider", min:1, max:200, step:1}
controlvideo.guidance_scale = 10  #@param {type:"slider", min:1, max:50, step:0.5}
controlvideo.smoother_stpes = "19,20"  #@param {type:"string"}
controlvideo.window_size = None  #@param {type:"raw"}
controlvideo.controlnet_conditioning_scale = 1.0  #@param {type:"slider", min:0, max:1, step:0.05}
controlvideo.seed = None  #@param {type:"raw"}

#@markdown ###**RealESRGAN**
realesrgan = config()
realesrgan.in_path = controlvideo.out_path
realesrgan.out_path = "../data/dataset/nerf/train/"  #@param {type:"string"}
realesrgan.outscale = 4.0  #@param {type:"slider", min:1, max:200, step:1}
realesrgan.tile = 192  #@param {type:"slider", min:1, max:200, step:1}
realesrgan.tile_pad = 16  #@param {type:"slider", min:1, max:200, step:1}
realesrgan.pre_pad = 16  #@param {type:"slider", min:1, max:200, step:1}
realesrgan.face_enhance = True  #@param {type:"boolean"}
realesrgan.fp32 = False  #@param {type:"boolean"}
realesrgan.gpu_id = 0  #@param {type:"slider", min:1, max:200, step:1}

### Inference

In [ ]:
import glob
import os
import pathlib
import PIL.Image
from src.utils.image_wrapper import *

def load_images():
    video_frames = []
    controlnet_conditions = []
    for filepath in sorted(
        glob.glob(os.path.combine(controlvideo.controlnet_conditioning_path, "*.png"))
    ):
        pl = pathlib.Path(filepath)
        video_frame_path = os.path.combine(controlvideo.out_path, pl.stem, ".png")
        video_frames.append(PIL.Image.open(video_frame_path))
        controlnet_conditions.append(PIL.Image.open(filepath))
    return (video_frames, controlnet_conditions)

def display_at_index(index, video_frames, controlnet_conditions):
    display(image_wrapper(video_frames[index], "pil").concatenate(
        image_wrapper(controlnet_conditions[index], "pil")
    ).to_pil())

In [ ]:
!python inference_controlvideo.py \ 
    --out_path {controlvideo.out_path} \ 
    --sd_repo {controlvideo.sd_repo} \
    --vae_repo {controlvideo.vae_repo} \ 
    --controlnet_repo {controlvideo.controlnet_repo} \ 
    --ifnet_path {controlvideo.ifnet_path} \
    --cache_dir {controlvideo.cache_dir} \
    --prompt {controlvideo.prompt} \
    --negative_prompt {controlvideo.negative_prompt} \
    --textual_inversion_path {controlvideo.textual_inversion_path} \
    --controlnet_conditioning_path {controlvideo.controlnet_conditioning_path} \
    --video_length {controlvideo.video_length} \
    --num_inference_steps {controlvideo.num_inference_steps} \
    --guidance_scale {controlvideo.guidance_scale} \
    --smoother_steps {controlvideo.smoother_steps} \
    --window_size {controlvideo.window_size} \
    --controlnet_conditioning_scale {controlvideo.controlnet_conditioning_scale} \ 
    --seed {controlvideo.seed} \

In [ ]:
import time
from IPython.display import clear_output

display_interval = 1

video_frames, controlnet_conditions = load_images()
length = len(video_frames)

for i in range(length):
    display_at_index(i, video_frames, controlnet_conditions)
    time.sleep(display_interval)
    if i < length - 1:
        clear_output(True)

In [ ]:
!python inference_realesrgan.py \
    --in_path {realesrgan.in_path} \
    --out_path {realesrgan.out_path} \
    --outscale {realesrgan.outscale} \
    --tile_pad {realesrgan.tile_pad} \
    --pre_pad {realesrgan.pre_pad} \
    --face_enhance {realesrgan.face_enhance} \
    --fp32 {realesrgan.fp32} \
    --gpu_id {realesrgan.gpu_id} \